In [2]:
!pip install faiss

     |████████████████████████████████| 4.7MB 59kB/s 


In [4]:
!pip install swigfaiss

ERROR: Could not find a version that satisfies the requirement swigfaiss (from versions: none)
ERROR: No matching distribution found for swigfaiss


In [21]:
!pip install sentence_transformers

In [0]:
"""
The system trains BERT on the SNLI + MultiNLI (AllNLI) dataset
with softmax loss function. At every 1000 training steps, the model is evaluated on the
STS benchmark dataset
"""
import math
import logging
from datetime import datetime
from pathlib import Path

from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import joblib
#import faiss
import numpy as np
import pandas as pd
from sentence_transformers import models, losses
from sentence_transformers import SentencesDataset, LoggingHandler, SentenceTransformer
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import *
from sentence_transformers.util import batch_to_device
from sentence_transformers.readers.InputExample import InputExample

In [0]:
#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
#### /print debug information to stdout

## Training

In [0]:
# Read the dataset
batch_size = 16
nli_reader = NLIDataReader('datasets/AllNLI')
sts_reader = STSDataReader('datasets/stsbenchmark')
train_num_labels = nli_reader.get_num_labels()
model_save_path = 'output/training_nli_bert-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

In [24]:
# Use BERT for mapping tokens to embeddings
# Using manually downloaded model data:
#word_embedding_model = models.BERT('../models/bert-base-multilingual-cased/')
# Or you can let the library handle the downloading and caching for you:
word_embedding_model = models.BERT('bert-base-multilingual-cased')

2020-01-11 09:57:41 - loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-config.json from cache at /root/.cache/torch/transformers/45629519f3117b89d89fd9c740073d8e4c1f0a70f9842476185100a8afe715d1.83b0fa3d7f1ac0e113ad300189a938c6f14d0588a4200f30eef109d0a047c484
2020-01-11 09:57:41 - Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads":

In [0]:
def children(m):
    return m if isinstance(m, (list, tuple)) else list(m.children())


def set_trainable_attr(m, b):
    m.trainable = b
    for p in m.parameters():
        p.requires_grad = b


def apply_leaf(m, f):
    c = children(m)
    if isinstance(m, nn.Module):
        f(m)
    if len(c) > 0:
        for l in c:
            apply_leaf(l, f)


def set_trainable(l, b):
    apply_leaf(l, lambda m: set_trainable_attr(m, b))

In [26]:
set_trainable(word_embedding_model.bert.embeddings.word_embeddings, False)
print(word_embedding_model.bert.embeddings.word_embeddings.weight.requires_grad)
print(word_embedding_model.bert.embeddings.position_embeddings.weight.requires_grad)

False
True


In [27]:
# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

2020-01-11 10:08:29 - Use pytorch device: cpu


In [28]:
# # Convert the dataset to a DataLoader ready for training
logging.info("Read AllNLI train dataset")
train_data = SentencesDataset(nli_reader.get_examples('train.gz'), model=model)
train_dataloader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
train_loss = losses.SoftmaxLoss(
    model=model, sentence_embedding_dimension=model.get_sentence_embedding_dimension(), num_labels=train_num_labels)

logging.info("Read STSbenchmark dev dataset")
dev_data = SentencesDataset(examples=sts_reader.get_examples('sts-dev.csv'), model=model)
dev_dataloader = DataLoader(dev_data, shuffle=False, batch_size=batch_size)
evaluator = EmbeddingSimilarityEvaluator(dev_dataloader)

2020-01-11 10:08:57 - Read AllNLI train dataset


FileNotFoundError: ignored

In [19]:
# joblib.dump(train_data, "allnli_train_dataset.jl")
# joblib.dump(dev_data, "sts_dev_dataset.jl")

NameError: ignored

In [17]:
# Convert the dataset to a DataLoader ready for training
#logging.info("Read AllNLI train dataset")
#train_data = joblib.load("allnli_train_dataset.jl")
#train_dataloader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
#train_loss = losses.SoftmaxLoss(
#    model=model, sentence_embedding_dimension=model.get_sentence_embedding_dimension(), num_labels=train_num_labels)

#logging.info("Read STSbenchmark dev dataset")
#dev_data = joblib.load("sts_dev_dataset.jl")
#dev_dataloader = DataLoader(dev_data, shuffle=False, batch_size=batch_size)
#evaluator = EmbeddingSimilarityEvaluator(dev_dataloader)

2020-01-11 09:46:02 - Read
2020-01-11 09:46:02 - Read
2020-01-11 09:46:02 - Read
2020-01-11 09:46:02 - Read
2020-01-11 09:46:02 - Read
2020-01-11 09:46:02 - Read AllNLI train dataset


FileNotFoundError: ignored

In [0]:
# Configure the training
num_epochs = 1

warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))

# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=model_save_path
          )

2019-09-22 21:51:44 - Warmup-steps: 5888


Convert Evaluating:   3%|▎         | 3/94 [00:00<00:03, 26.32it/s]

2019-09-22 21:56:24 - Evaluation the model on  dataset in epoch 0 after 1000 steps:


Convert Evaluating: 100%|██████████| 94/94 [00:05<00:00, 17.75it/s]


2019-09-22 21:56:30 - Cosine-Similarity :	Pearson: 0.5370	Spearman: 0.5462
2019-09-22 21:56:30 - Manhattan-Distance:	Pearson: 0.5615	Spearman: 0.5749
2019-09-22 21:56:30 - Euclidean-Distance:	Pearson: 0.5604	Spearman: 0.5736
2019-09-22 21:56:30 - Dot-Product-Similarity:	Pearson: 0.2355	Spearman: 0.2451
2019-09-22 21:56:30 - Save model to output/training_nli_bert-2019-09-22_21-50-05


...

2019-09-23 02:29:33 - Cosine-Similarity :	Pearson: 0.7573	Spearman: 0.7699
2019-09-23 02:29:33 - Manhattan-Distance:	Pearson: 0.7724	Spearman: 0.7723
2019-09-23 02:29:33 - Euclidean-Distance:	Pearson: 0.7717	Spearman: 0.7719
2019-09-23 02:29:33 - Dot-Product-Similarity:	Pearson: 0.7480	Spearman: 0.7538


Convert Evaluating:   3%|▎         | 3/94 [00:00<00:03, 28.32it/s]


2019-09-23 02:33:40 - Evaluation the model on  dataset after epoch 0:


Convert Evaluating: 100%|██████████| 94/94 [00:05<00:00, 17.85it/s]

2019-09-23 02:33:45 - Cosine-Similarity :	Pearson: 0.7573	Spearman: 0.7699
2019-09-23 02:33:45 - Manhattan-Distance:	Pearson: 0.7723	Spearman: 0.7721
2019-09-23 02:33:45 - Euclidean-Distance:	Pearson: 0.7717	Spearman: 0.7719
2019-09-23 02:33:45 - Dot-Product-Similarity:	Pearson: 0.7480	Spearman: 0.7540



In [0]:
model_save_path

'output/training_nli_bert-2019-09-22_21-50-05'

In [0]:
##############################################################################
#
# Load the stored model and evaluate its performance on STS benchmark dataset
#
##############################################################################

model = SentenceTransformer(model_save_path)
test_data = SentencesDataset(examples=sts_reader.get_examples("sts-test.csv"), model=model)
test_dataloader = DataLoader(test_data, shuffle=False, batch_size=batch_size)
evaluator = EmbeddingSimilarityEvaluator(test_dataloader)

model.evaluate(evaluator)

2019-09-23 14:37:07 - Load pretrained SentenceTransformer: output/training_nli_bert-2019-09-22_21-50-05
2019-09-23 14:37:07 - Load SentenceTransformer from folder: output/training_nli_bert-2019-09-22_21-50-05
2019-09-23 14:37:07 - loading configuration file output/training_nli_bert-2019-09-22_21-50-05/0_BERT/config.json
2019-09-23 14:37:07 - Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "torchscript": false,
  "type_vocab_size": 2,
  "

Convert Evaluating:   3%|▎         | 3/87 [00:00<00:03, 27.20it/s]

2019-09-23 14:37:10 - Num sentences: 1379
2019-09-23 14:37:10 - Sentences 0 longer than max_seqence_length: 0
2019-09-23 14:37:10 - Sentences 1 longer than max_seqence_length: 0
2019-09-23 14:37:10 - Evaluation the model on  dataset:


Convert Evaluating: 100%|██████████| 87/87 [00:04<00:00, 20.02it/s]

2019-09-23 14:37:14 - Cosine-Similarity :	Pearson: 0.6995	Spearman: 0.7248
2019-09-23 14:37:14 - Manhattan-Distance:	Pearson: 0.7311	Spearman: 0.7258
2019-09-23 14:37:14 - Euclidean-Distance:	Pearson: 0.7307	Spearman: 0.7259
2019-09-23 14:37:14 - Dot-Product-Similarity:	Pearson: 0.6929	Spearman: 0.7024


0.7259206832532674

## Tatoeba

In [0]:
class TatoebaReader:
    """Reads in a plain text file, in which every line contains one 
    sentence."""
    def __init__(self, file_path: Path):
        self.file_path = file_path

    def get_examples(self):
        examples = []        
        with open(self.file_path) as fin:
            for i, line in enumerate(fin.readlines()):
                examples.append(InputExample(guid=i, texts=[line], label=0))                
        return examples

TATOEBA_PATH = Path("../data/tatoeba/v1/")

In [0]:
def evaluate_language_pair(model, pair_name="cmn-eng", batch_size=32):
    lang_1, lang_2 = pair_name.split("-")
    reader_1 = TatoebaReader(TATOEBA_PATH / f"tatoeba.{pair_name}.{lang_1}")
    ds_1 = SentencesDataset(reader_1.get_examples(), model=model)
    loader_1 = DataLoader(
        ds_1, shuffle=False, batch_size=batch_size, 
        collate_fn=model.smart_batching_collate)

    reader_2 = TatoebaReader(TATOEBA_PATH / f"tatoeba.{pair_name}.{lang_2}")
    ds_2 = SentencesDataset(reader_2.get_examples(), model=model)
    loader_2 = DataLoader(
        ds_2, shuffle=False, batch_size=batch_size, 
        collate_fn=model.smart_batching_collate)
    
    model.eval()
    emb_1, emb_2 = [], []
    with torch.no_grad():
        for batch in loader_1:
            emb_1.append(model(
                batch_to_device(batch, "cuda")[0][0]
            )['sentence_embedding'])
        for batch in loader_2:
            emb_2.append(model(
                batch_to_device(batch, "cuda")[0][0]
            )['sentence_embedding'])
    emb_1 = torch.cat(emb_1).cpu().numpy()
    emb_2 = torch.cat(emb_2).cpu().numpy()
    
    idx_1 = faiss.IndexFlatL2(emb_1.shape[1])
    faiss.normalize_L2(emb_1)
    idx_1.add(emb_1)
    idx_2 = faiss.IndexFlatL2(emb_2.shape[1])
    faiss.normalize_L2(emb_2)
    idx_2.add(emb_2)
    
    results = []
    _, match = idx_2.search(x=emb_1, k=1)
    results.append((
        lang_1, lang_2,
        np.sum(match[:, 0] == np.arange(len(emb_1))),
        len(emb_1)
    ))
    _, match = idx_1.search(x=emb_2, k=1)
    results.append((
        lang_2, lang_1,
        np.sum(match[:, 0] == np.arange(len(emb_2))),
        len(emb_2)
    ))
    return results

In [0]:
PAIRS = ["ita-eng", "spa-eng", "fra-eng", "deu-eng", "rus-eng", "jpn-eng", "cmn-eng", "hin-eng"]

### Fine-tuned

In [0]:
model = SentenceTransformer('output/training_nli_bert-2019-09-22_21-50-05')

In [0]:
results = []
for pair in PAIRS:
    results += evaluate_language_pair(model, pair_name=pair, batch_size=50)
df_finetuned = pd.DataFrame(results, columns=["from", "to", "correct", "total"])

In [0]:
df_finetuned

,from,to,correct,total
0,ita,eng,498,1000
1,eng,ita,516,1000
2,spa,eng,532,1000
3,eng,spa,554,1000
4,fra,eng,511,1000
5,eng,fra,508,1000
6,deu,eng,513,1000
7,eng,deu,499,1000
8,rus,eng,498,1000
9,eng,rus,488,1000


## Baseline

In [0]:
word_embedding_model = models.BERT('../models/bert-base-multilingual-cased/')
# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [0]:
results = []
for pair in PAIRS:
    results += evaluate_language_pair(model, pair_name=pair, batch_size=50)
df_baseline_mean = pd.DataFrame(results, columns=["from", "to", "correct", "total"])
df_baseline_mean

,from,to,correct,total
0,ita,eng,500,1000
1,eng,ita,383,1000
2,spa,eng,521,1000
3,eng,spa,477,1000
4,fra,eng,482,1000
5,eng,fra,385,1000
6,deu,eng,488,1000
7,eng,deu,478,1000
8,rus,eng,430,1000
9,eng,rus,415,1000


In [0]:
word_embedding_model = models.BERT('../models/bert-base-multilingual-cased/')
# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=False,
                               pooling_mode_cls_token=True,
                               pooling_mode_max_tokens=False)
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [0]:
results = []
for pair in PAIRS:
    results += evaluate_language_pair(model, pair_name=pair, batch_size=50)
df_baseline_cls = pd.DataFrame(results, columns=["from", "to", "correct", "total"])

In [0]:
df_baseline_cls

,from,to,correct,total
0,ita,eng,121,1000
1,eng,ita,224,1000
2,spa,eng,109,1000
3,eng,spa,291,1000
4,fra,eng,169,1000
5,eng,fra,230,1000
6,deu,eng,115,1000
7,eng,deu,263,1000
8,rus,eng,165,1000
9,eng,rus,213,1000


In [0]:
word_embedding_model = models.BERT('../models/bert-base-multilingual-cased/')
# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=False,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=True)
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [0]:
results = []
for pair in PAIRS:
    results += evaluate_language_pair(model, pair_name=pair, batch_size=50)
df_baseline_max = pd.DataFrame(results, columns=["from", "to", "correct", "total"])

In [0]:
df_baseline_max

,from,to,correct,total
0,ita,eng,295,1000
1,eng,ita,373,1000
2,spa,eng,320,1000
3,eng,spa,427,1000
4,fra,eng,200,1000
5,eng,fra,335,1000
6,deu,eng,193,1000
7,eng,deu,363,1000
8,rus,eng,216,1000
9,eng,rus,320,1000


### Comparison

In [0]:
df_baseline_mean["err_mean"] = 1 - df_baseline_mean["correct"] / df_baseline_mean["total"]
df_baseline_max["err_max"] = 1 - df_baseline_max["correct"] / df_baseline_max["total"]
df_baseline_cls["err_cls"] = 1 - df_baseline_cls["correct"] / df_baseline_cls["total"]
df_finetuned["err_finetuned"] = 1 - df_finetuned["correct"] / df_finetuned["total"]

In [0]:
df_err = pd.concat([
    df.set_index(["from", "to"]).drop(["correct", "total"], axis=1)
    for df in (df_baseline_mean, df_baseline_max, df_baseline_cls, df_finetuned)
], axis=1)
df_err

,,err_mean,err_max,err_cls,err_finetuned
from,to,,,,
ita,eng,0.500,0.705,0.879,0.502
eng,ita,0.617,0.627,0.776,0.484
spa,eng,0.479,0.680,0.891,0.468
eng,spa,0.523,0.573,0.709,0.446
fra,eng,0.518,0.800,0.831,0.489
eng,fra,0.615,0.665,0.770,0.492
deu,eng,0.512,0.807,0.885,0.487
eng,deu,0.522,0.637,0.737,0.501
rus,eng,0.570,0.784,0.835,0.502


In [0]:
df_err["diff_mean"] = df_err["err_finetuned"] - df_err["err_mean"]
df_err["diff_pct_mean"] = df_err["diff_mean"] / df_err["err_mean"]
df_err["diff_max"] = df_err["err_finetuned"] - df_err["err_max"]
df_err["diff_pct_max"] = df_err["diff_max"] / df_err["err_max"]
df_err["diff_cls"] = df_err["err_finetuned"] - df_err["err_cls"]
df_err["diff_pct_cls"] = df_err["diff_cls"] / df_err["err_cls"]
df_err

,,err_mean,err_max,err_cls,err_finetuned,diff_mean,diff_max,diff_cls,diff_pct_mean,diff_pct_max,diff_pct_cls
from,to,,,,,,,,,,
ita,eng,0.500,0.705,0.879,0.502,0.002,-0.203,-0.377,0.004000,-0.287943,-0.428896
eng,ita,0.617,0.627,0.776,0.484,-0.133,-0.143,-0.292,-0.215559,-0.228070,-0.376289
spa,eng,0.479,0.680,0.891,0.468,-0.011,-0.212,-0.423,-0.022965,-0.311765,-0.474747
eng,spa,0.523,0.573,0.709,0.446,-0.077,-0.127,-0.263,-0.147228,-0.221640,-0.370945
fra,eng,0.518,0.800,0.831,0.489,-0.029,-0.311,-0.342,-0.055985,-0.388750,-0.411552
eng,fra,0.615,0.665,0.770,0.492,-0.123,-0.173,-0.278,-0.200000,-0.260150,-0.361039
deu,eng,0.512,0.807,0.885,0.487,-0.025,-0.320,-0.398,-0.048828,-0.396530,-0.449718
eng,deu,0.522,0.637,0.737,0.501,-0.021,-0.136,-0.236,-0.040230,-0.213501,-0.320217
rus,eng,0.570,0.784,0.835,0.502,-0.068,-0.282,-0.333,-0.119298,-0.359694,-0.398802


In [0]:
df_err.to_csv("df_err.csv")